In [34]:
from electric import solve
# from optic import solve
# from thermal import solve
# from degrading import solve

import numpy as np

In [57]:
class Boundary():
    def __init__(self,variable, condition_type, coefficients, external_normal, area):
        self.variable = variable
        self.condition_type = condition_type
        self.coefficients = coefficients
        self.area = area
        self.external_normal = external_normal

In [ ]:
def step_solver(old_T, old_g):
    new_I, new_Q = optic.solve(old_T, old_g)
    new_u, new_q = electric.solve(old_T, old_g)
    
    new_T = thermal.solve(old_T, new_Q)
    new_g = degrading.solve(old_g, new_T)
    
    return {"I": new_I, "Q": new_Q, "u": new_u, "q": new_q, "T": new_T, "g": new_g}

In [ ]:
def solver():
    solution = {
        "T": np.array([condition_start['T']]),
        "g": np.array([condition_start['g']])
    }
    for k in range(K):
        step_solution = step_solver(solution['T'][-1], solution['g'][-1])
    
        solution['I'] = np.concatenate((solution['I'], [step_solution['I']]))
        solution['Q'] = np.concatenate((solution['Q'], [step_solution['Q']]))
        solution['u'] = np.concatenate((solution['u'], [step_solution['u']]))
        solution['q'] = np.concatenate((solution['q'], [step_solution['q']]))
        solution['T'] = np.concatenate((solution['T'], [step_solution['T']]))
        solution['g'] = np.concatenate((solution['g'], [step_solution['g']]))

In [35]:
N = 100
dn = 10 ** (-9)
M = 200
dm = 10 ** (-9)
K = 10
dK = 10 ** (-6)

In [44]:
properties_electro = {
    "eps": lambda T,g: 81 * T/T,
    "sigma": lambda T,g: (10 ** (-1) * (1-g) + 10 ** (-3) * g) * (1-0.01 * (T-360)),
    "diffusion": lambda T,g: 10 ** (-7) * T
}

properties_thermo = {}

properties_optic = {}

In [ ]:
condition_start = {
    'T': 300 * np.ones((N,M)),
    'g': 1 * np.ones((N,M))
}

In [64]:
coo_n_start = np.array([[0, m] for m in range(M)])
coo_n_stop = np.array([[N - 1, m] for m in range(M)])
coo_m_start = np.array([[n, 0] for n in range(0, N)])
coo_m_stop = np.array([[n, M - 1] for n in range(0, N)])

def condition_boundary_termal(T,g):
    return [
        Boundary('T','Newton',coo_n_start,[-1,0],np.ones(M)),
        Boundary('T','Dirichlet',coo_n_stop,[1,0],np.zeros(M)),
        Boundary('T','Newmann',coo_m_start[1:-1],[0,-1],np.zeros(N-2)),
        Boundary('T','Dirichlet',coo_m_stop[1:-1],[0,1],np.zeros(N-2)),
    ]

def condition_boundary_optic(T,g):
    return [
        Boundary('I','Newton',coo_n_start,[-1,0],np.ones(M)),
        Boundary('I','Dirichlet',coo_n_stop,[1,0],np.zeros(M)),
        Boundary('I','Newmann',coo_m_start[1:-1],[0,-1],np.zeros(N-2)),
        Boundary('I','Dirichlet',coo_m_stop[1:-1],[0,1],np.zeros(N-2)),
    ]

NAir = N
coo_air_n_start = np.array([[0, m] for m in range(M)])
coo_air_n_stop = np.array([[NAir - 1, m] for m in range(M)])
coo_air_m_start = np.array([[n, 0] for n in range(0, NAir)])
coo_air_m_stop = np.array([[n, M - 1] for n in range(0, NAir)])

coo_tissue_n_start = np.array([[NAir, m] for m in range(M)])
coo_tissue_n_stop = np.array([[N + NAir - 1, m] for m in range(M)])
coo_tissue_m_start = np.array([[n, 0] for n in range(NAir, N + NAir)])
coo_tissue_m_stop = np.array([[n, M - 1] for n in range(NAir, N + NAir)])

# el1_M = int(7 * M / 15)
# el2_M = int(8 * M / 15)
# ne_M = np.delete(np.arange(M),[el1_M,el2_M])
# coo_air_el1 = np.array([[NAir - 1, el1_M]])
# coo_air_el2 = np.array([[NAir - 1, el2_M]])
# coo_air_n_stop_ne = np.array([[NAir - 1, m] for m in ne_M])  
# coo_tissue_el1 = np.array([[NAir, el1_M]])
# coo_tissue_el2 = np.array([[NAir, el2_M]])
# coo_tissue_n_start_ne = np.array([[NAir, m] for m in ne_M])  

def condition_boundary_electro(T,g): 
    return [
        Boundary('u','Dirichlet',coo_air_n_start,[-1,0],np.zeros(M)),
        Boundary('u','Dirichlet',coo_air_m_start[1:-1],[0,-1],np.zeros(NAir-2)),
        Boundary('u','Dirichlet',coo_air_m_stop[1:-1],[0,1],np.zeros(NAir-2)),

        Boundary('q','Dirichlet',coo_air_n_start,[-1,0],np.zeros(M)),
        Boundary('q','Dirichlet',coo_air_m_start[1:-1],[0,-1],np.zeros(NAir-2)),
        Boundary('q','Dirichlet',coo_air_m_stop[1:-1],[0,1],np.zeros(NAir-2)),

        Boundary('u','Newmann',coo_tissue_n_stop,[1,0],np.zeros(M)),
        Boundary('u','Dirichlet',coo_tissue_m_start[1:-1],[0,-1],np.ones(N-2)),
        Boundary('u','Dirichlet',coo_tissue_m_stop[1:-1],[0,1],-np.ones(N-2)),

        Boundary('j','Newmann',coo_tissue_n_stop,[1,0],np.zeros(M)),
        Boundary('j','Newmann',coo_tissue_m_start[1:-1],[0,-1],np.zeros(N-2)),
        Boundary('j','Newmann',coo_tissue_m_stop[1:-1],[0,1],np.zeros(N-2)),

        Boundary('u','Continiosly',[coo_air_n_stop,coo_tissue_n_start],[[0,1],[0,-1]],
                 [np.ones(M),properties_electro['eps'](T,g)]),
        Boundary('v','Dirichlet',coo_air_n_stop,[0,1],np.zeros(M)),
        Boundary('j','Newmann',coo_tissue_n_start,[-1,0],np.zeros(M))
    ]